# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation
from madminer.plotting import plot_fisher_information_contours_2d

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'
result_dir = base_dir + 'data/results/wgamma/'

## Settings

In [5]:
weight_settings = [None, 1., 10., 100.]
weight_setting_filenames = ['', '_w1', '_w10', '_w100']

## FisherInformation instances

In [8]:
fisher_all = FisherInformation(sample_dir + 'samples.h5', debug=True)
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5', debug=True)

10:33  
10:33  ------------------------------------------------------------
10:33  |                                                          |
10:33  |  MadMiner v2018.11.06                                    |
10:33  |                                                          |
10:33  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
10:33  |                                                          |
10:33  ------------------------------------------------------------
10:33  
10:33  Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
10:33  Found 2 parameters:
10:33     cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.1, 0.1))
10:33     cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.1, 0.1))
10:33  Found 6 benchmarks:
10:33     sm: cWWW = 0.00e+00, cWWWtilde = 0.00e+00
10:33     morphing_basis_vector_1: cWWW = -9.39e-02, cWWWtilde = 0.07
10:33     morphing_basis_vector_2: cWWW = 0.

## Truth-level info

In [ ]:
fisher_info, cov = fisher_all.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton.npy', cov)


In [ ]:
fisher_info, cov = fisher_tight.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_parton_tight.npy', cov)


## Rate-only info

In [ ]:
fisher_info, cov = fisher_all.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec.npy', cov)


In [ ]:
fisher_info, cov = fisher_tight.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec_tight.npy', fisher_info)
np.save(result_dir + 'information_covariance_xsec_tight.npy', cov)



## Full Fisher info (ML)

In [ ]:
means, covs = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full{}.npy'.format(weight_filename), cov)

10:54  Evaluating rate Fisher information
10:57  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/ensemble.json
10:57  Found ensemble with 10 estimators and expectations [[-0.00058942 -0.03821032]
 [-0.03163671  0.00532137]
 [-0.02699495  0.01435901]
 [ 0.01564022  0.00276239]
 [-0.02461902 -0.01322716]
 [ 0.05454716 -0.01634969]
 [-0.02868321 -0.04444946]
 [-0.06715274  0.00436619]
 [ 0.02157337 -0.01041408]
 [ 0.02201053 -0.01349001]]
10:57  Evaluating kinematic Fisher information on batch 1
10:57  Evaluating kinematic Fisher information on batch 2
10:58  Evaluating kinematic Fisher information on batch 3
10:58  Evaluating kinematic Fisher information on batch 4
10:59  Evaluating kinematic Fisher information on batch 5
10:59  Evaluating kinematic Fisher information on batch 6
10:59  Evaluating kinematic Fisher information on batch 7
11:00  Evaluating kinematic Fisher information on batch 8
11:00  Evaluating k

In [ ]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all_tight',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_full_tight{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_full_tight{}.npy'.format(weight_filename), cov)
    

## Minimal observable set (ML)

In [ ]:
means, covs = fisher_all.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_minimal',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_minimal{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_minimal{}.npy'.format(weight_filename), cov)

## Info in observables (ML)

In [ ]:
means, covs = fisher_tight.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_resurrection',
    unweighted_x_sample_file = sample_dir + 'test_tight/x_test.npy',
    ensemble_vote_expectation_weight=weight_settings
)

for weight_filename, mean, cov in zip(weight_setting_filenames, means, covs):
    np.save(result_dir + 'information_mean_resurrection{}.npy'.format(weight_filename), mean)
    np.save(result_dir + 'information_covariance_resurrection{}.npy'.format(weight_filename), cov)
    

## Histogram info

In [9]:
filenames = ['met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la', 'phi_resurrection']
observables = ['et_miss', 'pt_l1', 'pt_a1', 'deltaphi_l1_met', 'deltaphi_l1_a1', 'phi_resurrection']
bins = 25

In [10]:
for filename, obs in zip(filenames, observables):
    info, cov = fisher_tight.calculate_fisher_information_hist1d(
        theta=[0.,0.],
        luminosity=300000.,
        observable=obs,
        nbins=bins,
        histrange=None
    )
    
    np.save(result_dir + 'information_histo_{}_tight.npy'.format(filename), info)
    np.save(result_dir + 'information_covariance_histo_{}_tight.npy'.format(filename), cov)
    

10:33  Automatic dynamic binning: bin boundaries [  70.01706696   83.27401802   85.91010454   90.79949708   94.23582192
   98.52757233  102.20198377  105.95531917  110.36265613  115.93791004
  121.00044888  125.58452072  131.4259025   137.1404871   142.76535756
  149.03887361  157.35187023  166.80556495  177.01181918  189.59949433
  204.77676868  223.93111053  250.09795716  285.89578509  347.81225335
 3262.5949707 ]
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:931: RuntimeWarning: divide by zero encountered in true_divide
  inv_sigma = sanitize_array(1.0 / sigma)  # Shape (n_events,)
10:36  Automatic dynamic binning: bin boundaries [   70.01355957    86.44577406    93.0507576     98.57671777
   104.68688594   109.7128132    116.31964202   121.50834611
   125.84042444   132.06773663   140.21521879   147.3551914
   156.33388547   163.63033107   174.00087417   183.39140611
   195.3172811    212.51420422   232.5789048    253.87828034
   280.13675139   